In [8]:
import pandas as pd
import csv

name = 'dataset_tweet_2.csv'
data = open(name)
tokens = csv.reader(data, delimiter=';')
tweets = []; label = []
for row in tokens:
    tweets.append(row[0])
    label.append(int(row[1].replace(',','')))
df = pd.DataFrame(columns=['tweets','label'])
df['tweets'] = tweets
df['label'] = label
df

,tweets,label
0,"rt @napqilla: no 1, 3 ambisinya menguasai raky...",1
1,rt @pandji: nah gue pikir sentimen petahana ok...,1
2,rt @pandji: urutan pertama best moment #debat2...,1
3,rt @pandji: ini artikel yg menjelaskan ternyat...,1
4,rt @mrtampi: agus makin santai.\nahok makin sa...,0
...,...,...
76,rt @pandji: nah gue pikir sentimen petahana ok...,0
77,rt @josua_tm: ibu sylvi adalah contoh bahwa wa...,1
78,"besok saya ajak kesana saja, saya udah survei ...",1
79,benerr bgt.. dan tidak mengajak penonton ikut ...,1


In [17]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import re, string

clean_tweets=  []
for tweet in tweets:
    def hapus_tanda(tweet): 
        tanda_baca = set(string.punctuation)
        tweet = ''.join(ch for ch in tweet if ch not in tanda_baca)
        return tweet
    
    tweet=tweet.lower()
    tweet = re.sub(r'\\u\w\w\w\w', '', tweet)
    tweet=re.sub(r'http\S+','',tweet)
    #hapus @username
    tweet=re.sub('@[^\s]+','',tweet)
    #hapus #tagger 
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    #hapus tanda baca
    tweet=hapus_tanda(tweet)
    #hapus angka dan angka yang berada dalam string 
    tweet=re.sub(r'\w*\d\w*', '',tweet).strip()
    
    #stemming
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    tweet = stemmer.stem(tweet)
    clean_tweets.append(tweet)
df['cleaned'] = clean_tweets
df.head()

,tweets,label,cleaned
0,"rt @napqilla: no 1, 3 ambisinya menguasai raky...",1,rt no ambisi kuasa rakyat ambisi layan rakyat ...
1,rt @pandji: nah gue pikir sentimen petahana ok...,1,rt nah gue pikir sentimen tahana oke di malam ...
2,rt @pandji: urutan pertama best moment #debat2...,1,rt urut pertama best moment pak basuki misahin...
3,rt @pandji: ini artikel yg menjelaskan ternyat...,1,rt ini artikel yg jelas nyata di yg dapet resp...
4,rt @mrtampi: agus makin santai.\nahok makin sa...,0,rt agus makin santainahok makin santunnanies m...


In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import GaussianNB

vectorizer = TfidfVectorizer(max_features=2500)
model_g = GaussianNB()

v_data= vectorizer.fit_transform(df['cleaned'])
print(v_data)
v_data= vectorizer.fit_transform(df['cleaned']).toarray()

  (0, 10)	0.18913840804590096
  (0, 233)	0.27777504205235853
  (0, 263)	0.27777504205235853
  (0, 231)	0.23692725517145605
  (0, 170)	0.20394859337799942
  (0, 223)	0.27777504205235853
  (0, 339)	0.40789718675599884
  (0, 215)	0.27777504205235853
  (0, 15)	0.5555500841047171
  (0, 285)	0.27777504205235853
  (0, 346)	0.0754070869995221
  (1, 138)	0.28117693158765905
  (1, 45)	0.2624008048150012
  (1, 241)	0.2624008048150012
  (1, 328)	0.28117693158765905
  (1, 264)	0.28117693158765905
  (1, 403)	0.24783690942774547
  (1, 242)	0.28117693158765905
  (1, 110)	0.19637671509980026
  (1, 291)	0.28117693158765905
  (1, 397)	0.28117693158765905
  (1, 372)	0.28117693158765905
  (1, 320)	0.24783690942774547
  (1, 143)	0.24783690942774547
  (1, 274)	0.28117693158765905
  :	:
  (78, 351)	0.2026045906492247
  (78, 170)	0.19396503798408615
  (79, 256)	0.3893355655335795
  (79, 159)	0.3893355655335795
  (79, 422)	0.3893355655335795
  (79, 68)	0.3893355655335795
  (79, 61)	0.3893355655335795
  (79, 11)

In [33]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(v_data, df['label'], test_size=0.2, random_state=0)
model_g.fit(X_train,y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [34]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

y_preds = model_g.predict(X_test)

print(confusion_matrix(y_test,y_preds))
print(classification_report(y_test,y_preds))
print('nilai akurasinya adalah ',accuracy_score(y_test, y_preds))

[[8 4]
 [1 4]]
              precision    recall  f1-score   support

           0       0.89      0.67      0.76        12
           1       0.50      0.80      0.62         5

    accuracy                           0.71        17
   macro avg       0.69      0.73      0.69        17
weighted avg       0.77      0.71      0.72        17

nilai akurasinya adalah  0.7058823529411765


In [35]:
tweet=''
v_data= vectorizer.transform([tweet]).toarray()
y_preds = model_g.predict(v_data)
if y_preds ==1:
    print('positif')
else:
    print('negatif')

positif
